In [ ]:
# Make sure you have a GPU running
!nvidia-smi

Tue Feb 22 13:38:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Install the latest release of Haystack in your own environment
#! pip install farm-haystack

# Install the latest master of Haystack
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

# Install pygraphviz
!apt install libgraphviz-dev
!pip install pygraphviz

     |████████████████████████████████| 2.1 MB 8.5 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-rh3to90h/farm-haystack_917c273ccfc242d9b75c334ca9a5156b
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-rh3to90h/farm-haystack_917c273ccfc242d9b75c334ca9a5156b
  Resolved https://github.com/deepset-ai/haystack.git to commit d1b7761504e22f4d847ca3c58bd82f3042c8618c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.7/359.7 KB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.7/219.7 KB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [ ]:
# In Colab / No Docker environments: Start Elasticsearch from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT

es_server = Popen(
    ["elasticsearch-7.9.2/bin/elasticsearch"], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1)  # as daemon
)
# wait until ES has started
! sleep 30

In [ ]:
from haystack.utils import (
    print_answers,
    print_documents,
    fetch_archive_from_http,
    convert_files_to_dicts,
    clean_wiki_text,
)

# Download and prepare data - 517 Wikipedia articles for Game of Thrones
doc_dir = "data/article_txt_got"
s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt.zip"
fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

# convert files to dicts containing documents that can be indexed to our datastore
got_dicts = convert_files_to_dicts(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)

INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/
INFO - haystack.utils.import_utils -  Fetching from https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt.zip to `data/article_txt_got`
INFO - haystack.utils.preprocessing -  Converting data/article_txt_got/99_A_Song_of_Ass_and_Fire.txt
INFO - haystack.utils.preprocessing -  Converting data/article_txt_got/378_A_Game_of_Thrones__board_game_.txt
INFO - haystack.utils.preprocessing -  Converting data/article_txt_got/401_Power_Is_Power.txt
INFO - haystack.utils.preprocessing -  Converting data/article_txt_got/208_Robb_Stark.txt
INFO - haystack.utils.preprocessing -  Converting data/article_txt_got/340_Roose_Bolton.txt
INFO - haystack.utils.preprocessing -  Converting data/article_txt_got/201_A_Game_of_Thrones__card_game_.txt
INFO - haystack.utils.preprocessing -  Converting data/article_txt_got/51_Iron_Throne__A_Song_of_Ice_and_Fire_.

In [ ]:
from haystack import Pipeline
from haystack.utils import launch_es
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.nodes import ElasticsearchRetriever, DensePassageRetriever, FARMReader


# Initialize DocumentStore and index documents
launch_es()
document_store = ElasticsearchDocumentStore()
document_store.delete_documents()
document_store.write_documents(got_dicts)

# Initialize Sparse retriever
es_retriever = ElasticsearchRetriever(document_store=document_store)

# Initialize dense retriever
dpr_retriever = DensePassageRetriever(document_store)
document_store.update_embeddings(dpr_retriever, update_existing_embeddings=False)

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2")

WARNING - haystack.utils.doc_store -  Tried to start Elasticsearch through Docker but this failed. It is likely that there is already an existing Elasticsearch instance running. 
INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/493 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

WARNING - haystack.modeling.logger -  Failed to log params: Changing param values is not allowed. Param with key='processor' was already logged with value='TextSimilarityProcessor' for run ID='1ff94bdfd79e41949974725fb324bbf5'. Attempted logging new value 'SquadProcessor'.
INFO - haystack.modeling.logger -  ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.
INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.infer -  Got ya 2 parallel workers to do inference ...
INFO - haystack.modeling.infer -   0    0 
INFO - haystack.modeling.infer -  /w\  /w\
INFO - haystack.modeling.infer -  /'\  / \


In [ ]:
from haystack.pipelines import JoinDocuments

# Create ensembled pipeline
p_ensemble = Pipeline()
p_ensemble.add_node(component=es_retriever, name="ESRetriever", inputs=["Query"])
p_ensemble.add_node(component=dpr_retriever, name="DPRRetriever", inputs=["Query"])
p_ensemble.add_node(
    component=JoinDocuments(join_mode="concatenate"), name="JoinResults", inputs=["ESRetriever", "DPRRetriever"]
)
p_ensemble.add_node(component=reader, name="Reader", inputs=["JoinResults"])
p_ensemble.draw("pipeline_ensemble.png")

# Run pipeline
res = p_ensemble.run(
    query="Where is hometown of Arya Stark?", params={"DPRRetriever": {"top_k": 5}, "ESRetriever": {"top_k": 5}}
)
print_answers(res, details="minimum")

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]/usr/local/lib/python3.7/dist-packages/haystack/modeling/model/prediction_head.py:485: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 57.02 Batches/s]


Query: Where is hometown of Arya Stark?
Answers:
[   {   'answer': 'Winterfell',
        'context': ". Bran and Arya Stark, having heard of the Starks' "
                   'victory, return to Winterfell. Arya spars with Brienne and '
                   'takes Brienne aback when she is able to '},
    {   'answer': 'Winterfell',
        'context': 'aughter of Eddard Stark and Catelyn Stark. She was born '
                   'and raised in Winterfell, until leaving with her father '
                   'and sister at the beginning of the ser'},
    {   'answer': 'Winterfell',
        'context': 'g in the nearby town, Brienne manages to have a message '
                   'smuggled into Winterfell instructing Sansa how to signal '
                   'if she is in need of help. Although t'},
    {   'answer': 'Winterfell',
        'context': 'ut undeterred, she follows Sansa to the ancestral Stark '
                   'stronghold of Winterfell, where she has been wed to '
                

In [12]:
import random
painters = [
    "Ivan Shishkin", "Jim Burns", "Agnes Lawrence Pelton", "Amir Zand", "Ross Tran", "Edmund Dulac", "John Philip Falter", "Ron Walotsky", "Rembrandt van Rijn", 
    "Noah Bradley", "Arman Manookian", "Eyvind Earle", "Hilma af Klint", "Don Maitz", "René Magritte", "Tim White", "Raffaello Ossola", "Paul Lehr", "Geof Darrow", "Jean Giraud"
]

def make_prompt(query, painter=None):
    res = p_ensemble.run(
        query=query, params={"DPRRetriever": {"top_k": 5}, "ESRetriever": {"top_k": 5}}
    )
    print(res)
    print_answers(res, details="minimum")

    if painter is None:
        painter = random.choice(painters)
        
    res_list = res['answers'][0].to_dict()
    answer = res_list['answer']
    return f"a beautiful painting of {answer} by {painter}, Trending on artstation."

prompt = "Where is Jon Snow?"
print(f'Prompt: {make_prompt(prompt)}')

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]/usr/local/lib/python3.7/dist-packages/haystack/modeling/model/prediction_head.py:485: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 62.07 Batches/s]

{'query': 'Where is Jon Snow?', 'no_ans_gap': 7.558132171630859, 'answers': [<Answer {'answer': 'Wildling camp', 'type': 'extractive', 'score': 0.6761767119169235, 'context': "\n====Season 3====\nWhen Jon Snow first arrives in the Wildling camp, he initially mistakes Tormund for Mance Rayder, much to Tormund's amusement. Mance", 'offsets_in_document': [{'start': 53, 'end': 66}], 'offsets_in_context': [{'start': 53, 'end': 66}], 'document_id': 'ee7acf3af3540ce3b83c7024a82e473a', 'meta': {'name': '263_Tormund_Giantsbane.txt'}}>, <Answer {'answer': 'cave Grjótagjá', 'type': 'extractive', 'score': 0.5966198742389679, 'context': 's clearly seen. The nearby grotto where Jon and Ygritte have sex is cave Grjótagjá; however, the cave was used mainly for establishing shot of Jon Sno', 'offsets_in_document': [{'start': 589, 'end': 603}], 'offsets_in_context': [{'start': 68, 'end': 82}], 'document_id': '6670b17b54b841b53e05eb5b1715bf64', 'meta': {'name': '126_Kissed_by_Fire.txt'}}>, <Answer {'answ